<a href="https://colab.research.google.com/github/michelemiko1/genetic_music_CNN/blob/main/8__GA_classif_tuner_segments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hyperparameters tuner using Genetic Algorithms

In [ ]:
# import libraries
import tensorflow.keras as keras
from keras import layers
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
import json
import itertools

In [ ]:
# const values
DATASET_PATH = '/content/drive/MyDrive/Colab Notebooks/preprocessed_data_segments.json'
INPUT_SHAPE = (39, 130, 1)

NUM_GENERATIONS = 15
POPULATION = 10
PROB_CROSSOVER = 0.8
PROB_MUTATION = 0.2

In [ ]:
# load data from json file

def load_data(dataset_path):
    with open(dataset_path, 'r') as f:
        data = json.load(f)

    X_mfcc = np.array(data['MFCCs'])
    y = np.array(data['labels'])
    mapping = data['mapping']

    return X_mfcc, y, mapping

In [ ]:
# function to import the dataset

def split_dataset_segments(X, y):
  
    # all genres has 100 songs, but jazz has only 99 (1 was corrupted)
    max_songs_per_genre = 100 

    # 9 segments has been extracted for each song
    max_segments_per_genre = max_songs_per_genre * 9  # 900

    # keep 20% of samples for test set
    segments_test_per_genre = int(max_segments_per_genre/5)  # 180

    # IMPORTANT: I want to avoid to have segments of the same song divided into 
    # train and test, would be like cheating

    # save (almost) equal number of inputs for each genre in test set

    X_test = []
    y_test = []

    X_train = []
    y_train = []

    # consider all the labels 
    for i in range(10):
      
      count = 0

      # go through all the indexes
      for j in range(len(y)):
      
        # if the segment has the desired label and the number of segments is not reached
        if y[j] == i:

          if count < segments_test_per_genre:
            X_test.append(X[j])
            y_test.append(y[j])
            # print(f"label n: {i}, segment n: {count+1} appended to test set")
            count += 1
          else:
            X_train.append(X[j])
            y_train.append(y[j])
            # print(f"label n: {i}, segment n: DON'T KNOW appended to train set")

    # use np arrays
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)
  
    # add the channels dimension
    X_train = X_train[..., np.newaxis]
    X_test = X_test[..., np.newaxis]

    return (X_train, y_train), (X_test, y_test)

In [ ]:
# fuction to build the CNN model

def build_model(parameters, input_shape):

    '''
    # parameters values
    
    parameters[0] = number of Conv2D layers added (1,2,3)
    parameters[1] = number of Dense layers added  (1,2,3)
    parameters[2] = filters 1st Conv2D (min 32, max 128, step 16)
    parameters[3] = kernel  1st Conv2D(x) (3, 4, 5)
    parameters[4] = kernel  1st Conv2D(y) (3, 4, 5)
    parameters[5] = stride  1st Pool2D(x) (2, 3)
    parameters[6] = stride  1st Pool2D(y) (2, 3)

    parameters[7] = filters 2nd Conv2D (min 32, max 64, step 16)
    parameters[8] = filters 3rd Conv2D (min 32, max 64, step 16)
    parameters[9] = filters 4th Conv2D (min 32, max 64, step 16)
    parameters[10] = neurons 2nd Dense  (min 16, max 128, step 16)
    parameters[11] = neurons 3rd Dense  (min 16, max 128, step 16)
    parameters[12] = neurons 4th Dense  (min 16, max 128, step 16)
    parameters[13] = Dropout probability (0.1 - 0.5)

    '''

    model = keras.Sequential()

    # fist conv layer, max pooling and batch normalization

    model.add(layers.Conv2D(parameters[2], kernel_size=(parameters[3],parameters[4]),
                            input_shape=input_shape, activation='relu'))     
       
    model.add(layers.MaxPool2D(pool_size=(parameters[5], parameters[6]), padding='same')) 

    model.add(layers.BatchNormalization())

    # add other Conv layers max pooling and batch normalization

    for i in range(parameters[0]):
        model.add(layers.Conv2D(parameters[i+7], kernel_size=(3, 3), activation='relu', padding='same'))
        
        model.add(keras.layers.MaxPool2D((2, 2), strides=(2, 2), padding='same'))
        
        model.add(layers.BatchNormalization())


    # add flatten layer, dense layer and output (dense) layer
    model.add(layers.Flatten())
    
    # add Dense layers
    for i in range(parameters[1]):
        model.add(layers.Dense(units=parameters[i+10], activation='relu'))
        model.add(layers.Dropout(parameters[13]))

    # output layer
    model.add(layers.Dense(units=10, activation='softmax'))

    # compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
# perform the fitness of a single individual (CNN architecture)

def fitness_evaluate(individual, dataset):

    # show the individual
    print(f"\nIndividual considered: {individual}")

    # import dataset
    (X_train, y_train), (X_test, y_test) = dataset

    # find the input shape of our data. We cas simply use INPUT_SHAPE
    input_shape = np.shape(X_train[1])

    # create the model
    model = build_model(individual, input_shape)

    # train the model
    model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data = (X_test, y_test))

    # get the accuracy on test set
    print("\n")
    loss, accuracy = model.evaluate(X_test, y_test)

    return accuracy

In [ ]:
 # inizialize the fist population

def inizialize_population(pop_size=5):
  

    population = []

    # for each individual choose a random value in a specified range
    for i in range(pop_size):

      a = random.randint(1, 3)
      b = random.randint(1, 3)
      c = 16 * random.randint(2, 8)
      d = random.randint(3, 5)
      e = random.randint(3, 5)
      f = random.randint(2, 3)
      g = random.randint(2, 3)

      h = 16 * random.randint(2, 4)
      i = 16 * random.randint(2, 4)
      j = 16 * random.randint(2, 4)
      k = 16 * random.randint(1, 8)
      l = 16 * random.randint(1, 8)
      m = 16 * random.randint(1, 8)

      n = 0.1 + random.random() * 4 / 10  # number between 0.1 and 0.4
      
      individual = [a, b, c, d, e, f, g, h, i, j, k, l, m, n]

      # add the ith individual to the population
      population.append(individual)

    return population

In [ ]:
# evaluate the entire population and save results in a dictionary

def population_evaluation(population, dataset):

    # dictionary to store the fitness
    population_eval = {
        "population": [],   # [ [80, 5, 3, 48, 128], [128, 5, 3, 128, 112], ... ]
        "fitness": [],      # [ 0.77, 0.85, 0.54, ... ]
        "probability": []   # [ 0.21, 0.12, 0.51, ... ]
    }
    # save population into population_eval
    population_eval["population"] = population.copy()

    # for each individual (es: [2,4,67,1,2]) find the fitness and save it
    for individual in population:

        fitness = fitness_evaluate(individual, dataset)
        population_eval["fitness"].append(fitness)

    # calculate the probability of selection
    for fitness in population_eval["fitness"]:

        probability = fitness / np.sum(population_eval["fitness"])
        population_eval["probability"].append(probability)

    return population_eval

In [ ]:
# select two individuals to perform crossover

def selection(population_eval):

    population_size = len(population_eval["fitness"])
    selected_individual = []
    selected_individual2 = []

    # choose a random individual based on its probability
    condition = True
    while condition:

        # select a random value
        random_value = random.random()

        # select a random individual
        random_index = random.randint(0, population_size - 1)
        random_individual = population_eval["population"][random_index]

        # if his probability is major than a random value keep it
        prob_random_individual = population_eval["probability"][random_index]

        if prob_random_individual >= random_value:
            selected_individual = random_individual
            used_index = random_index
            condition = False

    # select another individual
    condition = True
    while condition:

        random_value = random.random()
        random_index = random.randint(0, population_size - 1)

        while random_index == used_index:
            random_index = random.randint(0, population_size - 1)

        random_individual = population_eval["population"][random_index]
        prob_random_individual = population_eval["probability"][random_index]

        if prob_random_individual >= random_value:
            selected_individual2 = random_individual
            condition = False

    return selected_individual, selected_individual2

In [ ]:
# crossover function between two individuals

def crossover(individual1, individual2, cross_probability=0.8):

    # define a crossover position
    len_individual = len(individual1)
    cross_position = random.randint(1, len_individual-1)

    # create the crossed individual
    cross_individual = individual1.copy()
    cross_individual[cross_position:] = individual2[cross_position:].copy()

    # use crossover probability
    if random.random() < cross_probability:
        return cross_individual
    else:
        return individual1.copy()

In [ ]:
def mutate_single_value(value, min, max, step):

  new_value = value + step * random.randint(-1, 1)

  # if it exceeds the boundaries approximate to the limit
  if new_value > max:
    new_value = max

  if new_value < min:
    new_value = min

  return new_value



In [ ]:
# mutation function 

def mutation(individual, prob_mutation=0.2):
 
    mut_individual = individual.copy()

    # apply the same probability of mutation for each value

    if random.random() < prob_mutation:
        mut_individual[0] = random.randint(1, 3)

    if random.random() < prob_mutation:
        mut_individual[1] = random.randint(1, 3)
    
    if random.random() < prob_mutation:

        # perform a random perturbation. If it exceeds give the max or min value
        old_value = mut_individual[2]
        mut_individual[2] = mutate_single_value(old_value, 32, 128, 16)

    
    if random.random() < prob_mutation:
        mut_individual[3] = random.randint(3, 5)

    if random.random() < prob_mutation:
        mut_individual[4] = random.randint(3, 5)

    if random.random() < prob_mutation:
        mut_individual[5] = random.randint(2, 3)

    if random.random() < prob_mutation:
        mut_individual[6] = random.randint(2, 3)



    if random.random() < prob_mutation:
        
        old_value = mut_individual[7]
        mut_individual[7] = mutate_single_value(old_value, 32, 64, 16)

    if random.random() < prob_mutation:
        
        old_value = mut_individual[8]
        mut_individual[8] = mutate_single_value(old_value, 32, 64, 16)

    if random.random() < prob_mutation:
        
        old_value = mut_individual[9]
        mut_individual[9] = mutate_single_value(old_value, 32, 64, 16)

    if random.random() < prob_mutation:
        
        old_value = mut_individual[10]
        mut_individual[10] = mutate_single_value(old_value, 16, 128, 16)

    if random.random() < prob_mutation:
        
        old_value = mut_individual[11]
        mut_individual[11] = mutate_single_value(old_value, 16, 128, 16)

    if random.random() < prob_mutation:
        
        old_value = mut_individual[12]
        mut_individual[12] = mutate_single_value(old_value, 16, 128, 16)


    if random.random() < prob_mutation:
        
        old_value = mut_individual[13]

        # set variation between -0.2 and 0.2
        variation = (random.random() - 0.5) / 2.5
        new_value = old_value + variation

        # limit the dropout probability to min 0.1 and max 0.5
        if new_value < 0.1:
          new_value = 0.1
        if new_value > 0.5:
          new_value = 0.5

        mut_individual[13] = new_value

    return mut_individual

In [ ]:
# program function that performs all the generations

def program():

    # import dataset
    X_mfcc, y, mapping = load_data(DATASET_PATH)

    # split dataset into train and test
    (X_train, y_train), (X_test, y_test) = split_dataset_segments(X_mfcc, y)
    dataset = (X_train, y_train), (X_test, y_test)

    # initialize the population
    population = inizialize_population(POPULATION)
    print("first population initialized\n")
    print(population)

    # initialize best global individual and best global fitness
    best_fitness_global = 0
    best_individual_global = []
    fitness_behaviour = []

    for gen in range(NUM_GENERATIONS):

        # calculate fitness and probabilities
        population_eval = population_evaluation(population, dataset)

        # save the best value of this generation
        best_fitness = np.max(population_eval["fitness"])
        best_index = population_eval["fitness"].index(best_fitness)
        best_individual = population_eval["population"][best_index]

        # keep track of best fitness of each generation
        fitness_behaviour.append(best_fitness)

        # save best global index and best global individual if we are 1st gen
        if gen == 0:
            best_fitness_global = best_fitness
            best_individual_global = best_individual

        # save best index and individual if it is better than previous generation
        if best_fitness > best_fitness_global:
            best_fitness_global = best_fitness
            best_individual_global = best_individual


        # CREATE THE NEW POPULATION - make selection, crossover and mutation until reach pop size

        if gen < NUM_GENERATIONS-1:
      
          new_population = []

          for i in range(POPULATION):

              # make the selection
              individual1, individual2 = selection(population_eval)

              # make crossover
              cross_individual = crossover(individual1, individual2, PROB_CROSSOVER)

              # make mutation
              new_individual = mutation(cross_individual, PROB_MUTATION)

              new_population.append(new_individual)

          population = new_population.copy()
          print(f"\npopulation number {gen + 2} created:")
          print(population)

    return best_fitness_global, best_individual_global, fitness_behaviour

In [ ]:
def plot_history(history):
  fig, axs = plt.subplots(2)

  # create the accuracy subplot
  axs[0].plot(history.history["accuracy"], label="train accuracy")
  axs[0].plot(history.history["val_accuracy"], label="test accuracy")
  axs[0].set_ylabel("Accuracy")
  axs[0].legend(loc="lower right")
  axs[0].set_title("Accuracy eval")

  # create the error subplot
  axs[1].plot(history.history["loss"], label="train error")
  axs[1].plot(history.history["val_loss"], label="test error")
  axs[1].set_ylabel("Error")
  axs[1].set_xlabel("Epoch")
  axs[1].legend(loc="upper right")
  axs[1].set_title("Error eval")

  plt.show()

In [ ]:
'''
def display_MFCCS(data, number_of_segments=999*9, hop_length=512, sr=22050):
    
    # select first index or each genre
    list_of_indexes = []
    for desired_label in range(10):
        for temporal_index in range(len(data['labels'])):

            #generate a random index between 0 and 999
            random_index = int(random.random() * number_of_songs)

            #verify if the corresponding label is equal to desired_label
            if data['labels'][random_index] == desired_label:
                list_of_indexes.append(random_index)
                break


    # mapping the labels
    labels = data['mapping']

    plt.figure(figsize=(12, 5))

    for i, index in enumerate(list_of_indexes):
        plt.subplot(2, 5, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        
        #select data
        data_array = np.array(data['MFCCs'][index])
        image_to_display = data_array

        # display MFCCs
        librosa.display.specshow(image_to_display, sr=SAMPLE_RATE, hop_length=hop_length)

        # extract and print the associated label
        current_label_index = data['labels'][index]
        current_label_name = labels[current_label_index]
        plt.xlabel(f"{current_label_name}\n ( sample:{index} )")


    plt.show()
'''

In [ ]:
# function used to plot the confusion matrix in a convenient way

def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# perform classification with the best CNN architecture obtained

def classification(individual, plot_conf_matrix=False):

  # build the model
  model = build_model(individual, INPUT_SHAPE)

  # import dataset
  X_mfcc, y, mapping = load_data(DATASET_PATH)

  # split dataset into train and test
  (X_train, y_train), (X_test, y_test) = split_dataset_segments(X_mfcc, y)

  # train the model
  history = model.fit(X_train, y_train, batch_size=16, epochs=15, validation_data=(X_test, y_test))

  # plot the results
  plot_history(history)

  # perform evaluation on test set
  loss, accuracy = model.evaluate(X_test, y_test)
  print(f"final model loss: {loss}, val_accuracy: {accuracy}")

  # plot confusion matrix
  if plot_conf_matrix:

    # get the predicted output and obtain the associated label
    predictions = model.predict(x = X_test, verbose=0)
    rounded_predictions = np.argmax(predictions, axis=-1)
    
    # compute the confusion matrix, define labels and plot
    conf_matrix = confusion_matrix(y_true=y_test, y_pred=rounded_predictions)
    cm_plot_labels = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
    plot_confusion_matrix(cm = conf_matrix, classes = cm_plot_labels, title='Confusion Matrix')

  return accuracy
  

In [ ]:
# run the program
best_fitness_global, best_individual_global, fitness_behaviour = program()

# print best results
print(f"\n BEST FITNESS: {best_fitness_global}")
print(f"\n BEST INDIVIDUAL: {best_individual_global}")
print(f"\n best fitness for each population: {fitness_behaviour}")


In [ ]:
# perform classification with best architecture ( ex: [3, 1, 112, 3, 5, 3, 2, 64, 64, 32, 64, 48, 80, 0.3741678893274713] )
print(best_individual_global)

# number of simulations considered
num_test = 3

# compute accuracy on test set and save it
test_accuracy = []
for i in range(num_test):
  accuracy = classification(best_individual_global, plot_conf_matrix=True)
  test_accuracy.append(accuracy)

# get the mean value for the accuracy
mean_accuracy_value = np.mean(test_accuracy)
print(f"the test accuracy is: {mean_accuracy_value}")



